In [21]:
""" Assumptions
  1.)using GC skew first, to find ori candidates
  then using k-mer enrichment to find exact ori

  2.)while adding antibiotic markers sequences, i don't think i would be able to include promoters for effective expression of the gene???
"""
while(1):
  break

In [22]:
!pip install biopython numpy
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
from collections import Counter

In [23]:
"""
  ORI finder code
"""
# 1. LOAD GENOME
record = SeqIO.read("Input.Fa", "fasta")
genome = str(record.seq).upper()
genome_len = len(genome)
print(f"Genome length: {genome_len}")

# 2. GC SKEW (FIND ROUGH ORI)
# Note: A simple cumulative sum of (G-C) is standard, but your ratio method also works.
# We will use your ratio method for consistency.
def gc_skew_calc(seq):
    g = seq.count("G")
    c = seq.count("C")
    return 0 if g + c == 0 else (g - c) / (g + c)

window = 1000
step = 200

skews = []
positions = []

for i in range(0, genome_len, step): # Removed "- window" to scan full length approx
    # Handle circular window for the very end of genome
    if i + window > genome_len:
        sub_seq = genome[i:] + genome[:(i + window) - genome_len]
    else:
        sub_seq = genome[i:i+window]

    skews.append(gc_skew_calc(sub_seq))
    positions.append((i + window//2) % genome_len)

skews = np.array(skews)
cum_skew = np.cumsum(skews)
ori_center_idx = np.argmin(cum_skew)
ori_center = positions[ori_center_idx]
print(f"Rough Skew Minimum (ori center): {ori_center}")

# 3. EXTRACT BROAD REGION (WITH CIRCULARITY)
broad_radius = 5000 # Reduced to 5kb (10kb total) to be more specific, 20kb is very wide
start = ori_center - broad_radius
end = ori_center + broad_radius

if start < 0:
    # Wrap around left side
    candidate_region = genome[start:] + genome[:end]
elif end > genome_len:
    # Wrap around right side
    candidate_region = genome[start:] + genome[:end-genome_len]
else:
    # Normal case
    candidate_region = genome[start:end]

print(f"Candidate region length: {len(candidate_region)}")

# 4. K-MER ENRICHMENT (WITH REVERSE COMPLEMENTS)
def get_canonical_kmer(kmer):
    """Returns the lexicographically first of kmer or its RC"""
    # This groups a sequence and its reverse complement together
    rc = str(Seq(kmer).reverse_complement())
    return min(kmer, rc)

def kmer_counts_canonical(seq, k):
    counts = Counter()
    for i in range(len(seq) - k + 1):
        kmer = seq[i:i+k]
        if "N" not in kmer:
            counts[get_canonical_kmer(kmer)] += 1
    return counts


k = 9
# Optimization: Only count genome kmers if really needed (can be slow).
# For assignment, it's fine.
genome_kmers = kmer_counts_canonical(genome, k)
candidate_kmers = kmer_counts_canonical(candidate_region, k)

# Normalize
for key in genome_kmers:
    genome_kmers[key] /= genome_len

for key in candidate_kmers:
    candidate_kmers[key] /= len(candidate_region)

# Enrichment score
enrichment = {
    kmer: candidate_kmers[kmer] / genome_kmers.get(kmer, 1e-9)
    for kmer in candidate_kmers
}

# Top 10 enriched DnaA boxes
top_kmers = sorted(enrichment.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top enriched motifs:", top_kmers)

# 5. PINPOINT FINAL ORI
window = 500 # Smaller window for fine-tuning
step = 50

scores = []
centers = []
top_set = {kmer for kmer, _ in top_kmers}

for i in range(0, len(candidate_region) - window, step):
    window_seq = candidate_region[i:i+window]
    # Check both forward and RC against our canonical list
    score = 0
    for j in range(len(window_seq) - k + 1):
        km = get_canonical_kmer(window_seq[j:j+k])
        if km in top_set:
            score += 1

    scores.append(score)
    centers.append(i + window//2)

best_local_idx = np.argmax(scores)
best_local_pos = centers[best_local_idx]

# Map back to absolute coordinates
# (This math is tricky with circularity, simplified here assuming we are inside the extracted region)
absolute_ori_center = (ori_center - broad_radius + best_local_pos) % genome_len

print(f"Refined ORI center: {absolute_ori_center}")

# Final Extraction
final_radius = 250
f_start = absolute_ori_center - final_radius
f_end = absolute_ori_center + final_radius

if f_start < 0:
    final_ori_seq = genome[f_start:] + genome[:f_end]
elif f_end > genome_len:
    final_ori_seq = genome[f_start:] + genome[:f_end-genome_len]
else:
    final_ori_seq = genome[f_start:f_end]

print("\nFinal ORI Sequence (First 50bp):")
print(final_ori_seq[:50] + "...")
print("final ori size", len(final_ori_seq))

Genome length: 2686
Rough Skew Minimum (ori center): 900
Candidate region length: 5372
Top enriched motifs: [('CTCTTCCGC', 186150.40953090097), ('CGGAAGAGC', 186150.40953090097), ('CGCTCTTCC', 186150.40953090097), ('GAAGAGCGC', 186150.40953090097), ('AAGAGCGCC', 186150.40953090097), ('AGAGCGCCC', 186150.40953090097), ('GAGCGCCCA', 186150.40953090097), ('AGCGCCCAA', 186150.40953090097), ('ATTGGGCGC', 1.0), ('CGCCCAATA', 1.0)]
Refined ORI center: 1036

Final ORI Sequence (First 50bp):
CCCCTATTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATG...
final ori size 500


In [24]:
"""
Adding Design.txt requirements
"""
import pandas as pd
from Bio import Entrez, SeqIO
import time
Entrez.email = "sang25wan25@gmail.com"

In [25]:
# Helper function - to get the ori of a plasmid given in design.txt using NCBI search but this is failing, so i created a local ori fasta library
def get_plasmid_ori_sequence(ori_name, ori_csv="plasmid_ori.csv"):
    # Load CSV
    try:
        df = pd.read_csv(ori_csv)
    except Exception as e:
        print(f"Failed to load {ori_csv}: {e}")
        return ""

    # Build mapping
    ori_dict = dict(zip(df["ori_name"], df["host_microorganism"]))

    host = ori_dict.get(ori_name)

    if not host:
        print(f"[FAIL] No host found for {ori_name} in CSV")
        return ""

    print(f"Searching ori '{ori_name}' in host '{host}'")

    query = f"{ori_name}[All Fields] AND {host}[Organism] AND plasmid[Title]"

    try:
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=1)
        record = Entrez.read(handle)
        handle.close()
    except Exception as e:
        print(f"[FAIL] NCBI search error: {e}")
        return ""

    if not record["IdList"]:
        print(f"[FAIL] No NCBI results for {ori_name} in {host}")
        return ""

    print("[PASS] NCBI search returned a result")

    seq_id = record["IdList"][0]

    try:
        handle = Entrez.efetch(
            db="nucleotide",
            id=seq_id,
            rettype="gb",
            retmode="text"
        )
        gb_record = SeqIO.read(handle, "genbank")
        handle.close()
    except Exception as e:
        print(f"[FAIL] Fetch error: {e}")
        return ""

    # Extract rep_origin
    for feature in gb_record.features:
        if feature.type == "rep_origin":
            print("[PASS] rep_origin found")
            return str(feature.extract(gb_record.seq))

    print("[FAIL] rep_origin not found in record")
    return ""

In [26]:
#local ori fasta library
def load_ori_library(fasta_file="plasmid_ori.fasta"):
    ori_dict = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        ori_dict[record.id] = str(record.seq)
    return ori_dict

In [27]:
# Helper function - Fetching sequences other than oris and restrcitions sites, that are mentioned in design.txt

def get_antibiotic_sequence(marker_name, marker_file="/content/markers.tab"):
    """
    Uses markers.tab to map abstract marker names to real gene identifiers
    before querying NCBI.
    """
    try:
        print(f"   ...Resolving marker: {marker_name}")

        # --- Load marker mappings ---
        marker_map = {}
        with open(marker_file, "r") as f:
            for line in f:
                if not line.strip() or line.startswith("#"):
                    continue
                parts = line.strip().split("\t")
                if len(parts) >= 2:
                    marker_map[parts[0].strip()] = parts[1].strip()

        # Resolve to actual gene / feature name
        search_term = marker_map.get(marker_name, marker_name)

        print(f"   ...Searching NCBI for '{search_term}'")

        query = f"{search_term}[Title] AND bacteria[Organism]"

        handle = Entrez.esearch(db="nucleotide", term=query, retmax=1)
        record = Entrez.read(handle)
        handle.close()

        if not record["IdList"]:
            print(f"    No entry found for {marker_name} ({search_term})")
            return ""

        seq_id = record["IdList"][0]

        handle = Entrez.efetch(
            db="nucleotide",
            id=seq_id,
            rettype="fasta",
            retmode="text"
        )
        seq_record = SeqIO.read(handle, "fasta")
        handle.close()

        print(f"   Found: {seq_record.description[:60]}...")
        return str(seq_record.seq)

    except Exception as e:
        print(f"   Error fetching from NCBI: {e}")
        return ""

In [28]:
# Main function - Consructing requirements of design.txt

def construct_plasmid_part(design_file, enzyme_csv):
    part1 = ""

    # 1. Load Restriction Enzymes into a Dictionary for fast lookup
    try:
        df = pd.read_csv(enzyme_csv)
        # Create a dictionary: {'EcoRI': 'GAATTC', ...}
        # Using strip() to remove accidental whitespace
        enzyme_dict = dict(zip(df.iloc[:, 0].str.strip(), df.iloc[:, 1].str.strip()))
    except FileNotFoundError:
        return "Error: restriction_enzymes.csv not found."

    # 2. Process Design File
    try:
        with open(design_file, 'r') as f:
            lines = f.readlines()
    except FileNotFoundError:
        return "Error: Design.txt not found."

    print("--- Starting Plasmid Construction ---")

    for line in lines:
        line = line.strip()
        if not line: continue # Skip empty lines

        # Split by comma
        parts = line.split(',')
        if len(parts) < 2: continue # Skip malformed lines

        key_type = parts[0].strip()
        value_name = parts[1].strip()

        # LOGIC CHECK
        if "_site" in key_type:
            # It is a restriction enzyme
            seq = enzyme_dict.get(value_name)
            if seq:
                print(f"➕ Adding Site: {value_name} ({seq})")
                part1 += seq
            else:
                print(f" Warning: Enzyme {value_name} not found in CSV.")
        else:
            # It is not a restriction enzyme, probably need internet to lookup the gene sequences.
            if "ori" in key_type.lower():
                ori_db = load_ori_library()
                temp = ori_db.get(key_type, "")
                part1 += temp
                continue
            gene_seq = get_antibiotic_sequence(key_type)
            part1 += gene_seq
            # Sleep briefly to be nice to NCBI servers
            time.sleep(1)

    print("--- Construction Complete ---")
    return part1
part1 = construct_plasmid_part("Design.txt", "restriction_enzymes.csv")

--- Starting Plasmid Construction ---
➕ Adding Site: BamHI (GGATCC)
➕ Adding Site: HindIII (AAGCTT)
➕ Adding Site: PstI (CTGCAG)
➕ Adding Site: SphI (GCATGC)
➕ Adding Site: SalI (GTCGAC)
➕ Adding Site: XbaI (TCTAGA)
➕ Adding Site: KpnI (GGTACC)
➕ Adding Site: SacI (GAGCTC)
➕ Adding Site: SmaI (CCCGGG)
   ...Resolving marker: AmpR_gene
   ...Searching NCBI for 'AmpR_gene'
   Found: MT293230.1 Pseudomonas aeruginosa strain ISP50 AmpR (ampR) g...
   ...Resolving marker: lacZ_alpha
   ...Searching NCBI for 'lacZ_alpha'
   Found: M74750.1 UNVERIFIED_ORG: E.coli LacZ-alpha peptide and beta-...
--- Construction Complete ---


In [29]:
"""
 IncQ plasmids have a broader host range
than any other known replicating element in bacteria.

Plasmid RSF1010 (8684 bp) contains three novel genes:
repA, repB, and repC. The product of the repA gene has
ssDNA-dependent ATPase and DNA helicase activity
(Scherzinger et al., 1997), repC product binds to the ite-
rons and opens the origin region, creating the entry site
for the RepA helicase (Scherzinger et al., 1991), and the
repB product encodes a primase. In vivo, a 2.1-kilobase
segment of the plasmid, bearing the replication origin,
can establish itself as an autonomous replicon if the DNA
region carrying the three rep genes is present in the same
cell on an independent plasmid (Scherzinger et al., 1984).
"""

'\n IncQ plasmids have a broader host range\nthan any other known replicating element in bacteria.\n\nPlasmid RSF1010 (8684 bp) contains three novel genes:\nrepA, repB, and repC. The product of the repA gene has\nssDNA-dependent ATPase and DNA helicase activity\n(Scherzinger et al., 1997), repC product binds to the ite-\nrons and opens the origin region, creating the entry site\nfor the RepA helicase (Scherzinger et al., 1991), and the\nrepB product encodes a primase. In vivo, a 2.1-kilobase\nsegment of the plasmid, bearing the replication origin,\ncan establish itself as an autonomous replicon if the DNA\nregion carrying the three rep genes is present in the same\ncell on an independent plasmid (Scherzinger et al., 1984).\n'

In [30]:
from Bio import Entrez, SeqIO

Entrez.email = "sang25wan25@gmail.com"  # <--- Update this

def get_incq_replicon():
    print("Fetching RSF1010 (M28829.1)...")
    try:
        handle = Entrez.efetch(db="nucleotide", id="M28829.1", rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
    except Exception as e:
        return f"Error fetching: {e}"

    print(f"   > Accession: {record.id} ({len(record.seq)} bp)")

    # --- 1. IDENTIFY THE REP GENE CLUSTER ---
    # We look for the "Trans" acting proteins (RepA, RepB, RepC).
    # In RSF1010, these form an operon.

    rep_genes_coords = []
    target_names = ["repA", "repB", "repC", "replication protein"]

    for feature in record.features:
        if feature.type == "CDS":
            # Search all qualifiers (product, gene, note) for keywords
            qualifier_text = str(feature.qualifiers).lower()

            if any(name in qualifier_text for name in target_names):
                start = int(feature.location.start)
                end = int(feature.location.end)
                rep_genes_coords.append((start, end))

                # Log what we found
                name = feature.qualifiers.get('gene', feature.qualifiers.get('product', ['Unknown']))[0]
                print(f"   > Found Rep Component: {name} ({start}..{end})")

    if not rep_genes_coords:
        return "Error: No Rep genes found. Check accession or keywords."

    # --- 2. DEFINE THE ATOMIC REPLICON SPAN ---
    # The IncQ replicon is ONE unit: [Upstream Origin/Iterons] -> [Promoters] -> [RepC-A-B]

    # A. Find the extent of the coding regions
    min_start = min(s for s, e in rep_genes_coords)
    max_end = max(e for s, e in rep_genes_coords)
    print(f"   > Gene Cluster Span: {min_start} to {max_end}")

    # B. Apply the "Safety Buffer" (The Critical Fix)
    # The RSF1010 Origin (oriV) is located roughly 700-1000 bp UPSTREAM of the first gene (repC).
    # We extend the selection upstream by 1500 bp to guarantee we capture:
    #   1. The oriV iterons
    #   2. The promoters (P2/P3) that drive Rep gene expression

    upstream_buffer = 1500  # Generous buffer to capture oriV
    downstream_buffer = 100 # Safety margin for terminator

    final_start = max(0, min_start - upstream_buffer)
    final_end = min(len(record.seq), max_end + downstream_buffer)

    # --- 3. EXTRACT THE SINGLE CONTIGUOUS BLOCK ---
    replicon_seq = record.seq[final_start:final_end]

    print(f"\n EXTRACTED Atomic IncQ Replicon")
    print(f"   > Coordinates: {final_start} to {final_end}")
    print(f"   > Total Length: {len(replicon_seq)} bp")
    print(f"   > Structure: [Origin region + Promoters + RepABC Operon]")

    return str(replicon_seq)

# --- EXECUTION ---
incq_module = get_incq_replicon()

part1 += incq_module
with open("output.txt", "w") as f:
    f.write(str(part1))

Fetching RSF1010 (M28829.1)...
   > Accession: M28829.1 (8684 bp)
   > Found Rep Component: replication protein B (4407..5379)
   > Found Rep Component: replication protein A (5889..6729)
   > Found Rep Component: replication protein C (6715..7567)
   > Gene Cluster Span: 4407 to 7567

 EXTRACTED Atomic IncQ Replicon
   > Coordinates: 2907 to 7667
   > Total Length: 4760 bp
   > Structure: [Origin region + Promoters + RepABC Operon]
